In [ ]:
# 불러올때
with open('data/result.txt', 'rb') as f:
    result = pickle.load(f)

# ---(박사님 같이) 코사인유사도 구하기 연습------

In [ ]:
test_m = np.array([1,2, 3, 4, 5])
test_m

In [ ]:
test_m = np.diag(test_m)
test_m

In [ ]:
Q = np.array([1,0,0,0, 0])
Q

In [ ]:
Q.reshape(5, -1)

In [ ]:
dot_pro.shape

In [ ]:
dot_pro

In [ ]:
Q.shape

In [ ]:
Q

In [ ]:
dot_pro = test_m.dot(Q)

In [ ]:
np.linalg.norm(test_m, axis=1)#길이

In [ ]:
np.linalg.norm(Q) #길이

In [ ]:
np.linalg.norm(test_m, axis=1) * np.linalg.norm(Q) #길

In [ ]:
sim = dot_pro / np.linalg.norm(test_m, axis=1) * np.linalg.norm(Q) #길

In [ ]:
sim.argmax()

# --------------

### -----문장간 코사인유사도 찾기(좋은 임베딩 파라미터 찾기------

In [ ]:
result[:10]

In [ ]:
#ebs, google 문장들 쪼개서
result_gensim_input = [_.split() for _ in result if _ != '']
print(len(result_gensim_input))
print(result_gensim_input[:3])

In [ ]:
#ebs 문장들 안쪼개서
result_tokenizer_input = [v for i, v in enumerate(result) if i%2 == 0 and v != '']
print(len(result_tokenizer_input))
print(result_tokenizer_input[:3])

In [ ]:
#ebs 문장들 쪼개서
ebs_in_result_for_getting_max_len = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != []]
print(len(ebs_in_result_for_getting_max_len))
print(ebs_in_result_for_getting_max_len[:3])

In [ ]:
ebs_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != [] ]
print(len(ebs_in_result_gensim_input))
print(ebs_in_result_gensim_input[:3])

In [ ]:
google_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 1 and v != []]

In [ ]:
embedding_size = 10
window_size = 2

model = Word2Vec(
                result_gensim_input, 
                window = window_size,
                size =embedding_size,
                min_count=1,
                workers = 10
                )

def get_s_vec(s_list, embedding_size):
    s_vec = np.zeros(embedding_size)
    for t in s_list:
        s_vec += model[t]
    s_vec /= len(s_list)
    return s_vec

In [ ]:
# Generate EBS string vectors matrix
ebs_str_arr = np.zeros((len(ebs_in_result_gensim_input), embedding_size))
for i, s_list in enumerate(ebs_in_result_gensim_input):
    ebs_str_arr[i] = get_s_vec(s_list)
ebs_vec_norms = np.linalg.norm(ebs_str_arr, axis=1)

In [ ]:
#Query comparison
cos_sims_bool = np.zeros(len(google_in_result_gensim_input))
for g_idx, Q_str in enumerate(google_in_result_gensim_input):
    Q_vec = get_s_vec(Q_str)
#     Q_vec = Q_vec.reshape(embedding_size, -1)
    dot_products = ebs_str_arr.dot(Q_vec)

    Q_vec_norm = np.linalg.norm(Q_vec)
    norm_productions = ebs_vec_norms * Q_vec_norm
    cos_sims = dot_products / norm_productions
    
#     print(g_idx, '   ', cos_sims.argmax())
    
    if g_idx == cos_sims.argmax():
        cos_sims_bool[g_idx] = 1
        
diff_num = np.unique(cos_sims_bool, return_counts=True)[1][0]
same_num = np.unique(cos_sims_bool, return_counts=True)[1][1]
print(same_num/(diff_num+same_num) * 100)